# データの読込・閲覧

## 準備（必要なパッケージ、データの読込）

In [ ]:
### 必要なパッケージ（ライブラリ）の読み込み ###
import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

In [ ]:
# データの読み込み
df = pd.read_excel('ファイル名入力（Tabキーによる文字補完を活用）')

# ※ "演習01_" まで入力し、TABキーで補完するのが効率的

## データ閲覧

In [ ]:
df

In [ ]:
df.head(3)

## 行数・列数の出力

In [ ]:
df.shape

# (復習)データの確認・各種統計量の観察

## 要約統計量の出力

In [ ]:
df.describe()

In [ ]:
### 参考：カテゴリー値の場合 ###
df.describe(include='O')   #'Object'型の頭文字'O'

## 相関行列

In [ ]:
### 相関行列 ###
df.corr(numeric_only=True) 

# pandasの仕様変更(おそらくv2.0.0以降）により、numeric_only=Trueの指定が必須化
# 指定しないと、文字列を含む列がある場合、"could not convert string to float...." というエラーが発生
# （従来は、当該引数を指定しなくても、自動的に文字列を含む列は除外されていた）
# 
# 参考 pandasのversion確認方法：print(pd.__version__)

In [ ]:
### 相関行列のお化粧（ヒートマップ） ###

import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）

# カラーマップの設定 (RdBu:赤〜青 ⇔ RdBu_r:青〜赤)
colormap = plt.cm.RdBu_r

# Figureサイズの指定
plt.rcParams['figure.figsize'] = 10, 5

# 相関行列の右上半分を隠したい場合は、次の2行のコメントを解除し、sns.heatmapのオプションに mask=mask を追加
# mask = np.zeros_like(df.corr(), dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

sns.heatmap(df.corr(numeric_only=True), linewidths=0.1, linecolor='white',   # 相関行列df.corr()を引数としヒートマップ作成
            vmax=1.0, vmin=-1.0, cmap=colormap, annot=True, annot_kws={'fontsize':8}, fmt='.2f')

            # linewidths/linecolor: 格子線の太さ/色
            # cmap:                 カラーマップの指定
            # vmax/vmin:            最大値/最小値
            # annot:                各要素への数値表示
            # annot_kws={'fontsize':8}: 各要素内に表示する数値の文字サイズを指定
            # fmt='.2f':            各要素内に表示する数値の小数点桁数を指定（'.2f'で小数第二位まで）
            # mask:                 可視化から除外する対象（例：mask=mask）
            

## クロス集計（カテゴリー値に有効）
クロス集計・・・2つの変数を「クロス」させて（掛け合わせて）、カテゴリー値の出現値（クラス）ごとの傾向を確認する方法  
`pd.crosstab(行方向に配置する変数, 列方向に配置する変数)`

In [ ]:
### クロス集計 ###
# チケットクラスごとの生存者数を確認
pd.crosstab(df['Pclass'], df['Survived'])  #pd.crosstab(行方向に配置する変数, 列方向に配置する変数)

In [ ]:
### クロス集計 <値正規化バージョン> ###
# チケットクラスごとの生存率を確認
pd.crosstab(df['Pclass'], df['Survived'], normalize='index')  #normalize='index'で行ごとの割合で集計される

In [ ]:
### クロス集計 <応用編> ###
# チケットクラス、生存ごとの平均年齢
pd.crosstab(df['Pclass'], df['Survived'], values=df['Age'], aggfunc='mean')  #values, aggfuncを指定すれば、他列の集計値を出力可能

### 参考：クロス集計表で値の大小に応じて色をつける方法

In [ ]:
# 一旦、クロス集計結果を別のデータフレームに格納
df_cross = pd.crosstab(df['Pclass'], df['Survived'], normalize='index') 

# データフレームのスタイル機能を活用し、行ごとの値の大小に応じてグラデーション
df_cross.style.background_gradient(axis=0, cmap='Reds')
     # axis: 大小を判断する範囲を指定（axis=Noneで全体、axis=0で行ごと、axis=1で列ごと）
     # cmap: カラーマップの指定

参考：カラーマップリスト
https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

## 欠損値の確認

In [ ]:
df.isnull().sum()

## 欠損値の処理 → 後段の分析時に対応

In [ ]:
# ### 欠損値を中央値で置換 ###
# df['Age'] = df['Age'].fillna(df['Age'].median())  #連続値は中央値／平均値
# df['Embarked'] = df['Embarked'].fillna('S')       #カテゴリー値は最頻値

# Cabinはひとまず除外して分析するため、一旦放置

In [ ]:
# # 欠損値処理後の確認
# df.isnull().sum()

# (復習)グラフによる可視化① - ヒストグラム

## 単一ヒストグラム

In [ ]:
sns.histplot( df['Fare'], kde=False, bins=10 )

## 複数のヒストグラム一括描画

In [ ]:
# pandasパッケージのプロット機能活用
df.hist()

plt.tight_layout()                                 # グラフ同士が重ならないようにするオプション

## 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、デザイン設定 ###
plt.rcParams['figure.figsize'] = 10, 5             # グラフのサイズ指定
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
                                                   # 文字化け対策にフォント指定
plt.rcParams['font.size'] = 16                     # フォントサイズ一括指定
plt.tight_layout()                                 # グラフ同士が重ならないようにする
# plt.grid(True)                                   # グリッド表示ON
plt.show()                                         # 各種設定の反映

# (復習)グラフによる可視化② - 散布図・箱ひげ図

## 散布図（単一）

In [ ]:
### 散布図（単一） ###	
sns.jointplot(data=df, x='Age', y='Fare')  # dfの「Age」と「Fare」のデータ列の散布図(jointplot)を描画

## 散布図（複数）→ 「散布図行列」

In [ ]:
# 非常に時間がかかるので注意
sns.pairplot(df)

## 生存フラグで色分けした散布図

In [ ]:
# 非常に時間がかかるので注意
sns.pairplot(df, hue= 'Survived' )

## 箱ひげ図

In [ ]:
# pandasパッケージのプロット機能活用
df.boxplot()

# 演習4-1 ロジスティック回帰

## データの準備（加工）

### 欠損値の補完

In [ ]:
### 欠損値の確認 ###
df.isnull().sum()

In [ ]:
### 欠損値の補完 ###
df['Age'] =      df['Age'].fillna(df['Age'].median()) # 連続数は中央値／平均値
df['Embarked'] = df['Embarked'].fillna( 'S' )         # カテゴリー値は最頻値 → Emberkedの最頻値'S' を代入
                                                      # （関数で与えたい場合には、df['Embarked'].mode()[0] ）

In [ ]:
df.isnull().sum()

### ダミー変数化

In [ ]:
### 「Sex」の「male／female」をダミー変数化（0/1） ###
df['Sex'] = df['Sex'].replace( {'male':0, 'female':1} ) # Noはただの識別番号のため可視化から除外

### 「Embarked」の「C/Q/S」をダミー変数化（0/1/2） ###
df['Embarked'] = df['Embarked'].replace( {'C':0, 'Q':1, 'S':2} ) # Noはただの識別番号のため可視化から除外

In [ ]:
df

## モデルの構築

In [ ]:
### ロジスティック回帰分析 ###
from sklearn import linear_model # 機械学習パッケージに含まれた回帰分析モジュール"linear_model"を読込

# 説明変数のセット
X = df[ ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] ]

# 目的変数のセット
Y = df['Survived']

# ロジスティック回帰モデルを作成
model = linear_model.LogisticRegression()  # インスタンス化（関数を使える状態にする）
model.fit(X, Y)                            # モデル構築（フィッティング）

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_.reshape(-1,)}) )
#もしくは
#display( pd.concat( [pd.DataFrame(X.columns, columns=['Variables']), pd.DataFrame(model.coef_.T, columns=['Coefficients'])], axis=1) )

# 切片の出力
print( '切片:', model.intercept_ )

# 正解率の出力（全データのうち、正解した件数の割合）
print( '正解率:', model.score(X, Y) )

## モデルの評価

### 予測結果の確認

In [ ]:
### 実測値と予測結果の比較 ###

Y_pred = model.predict(X)  # モデルに説明変数を入力して、予測結果を出力
results = pd.DataFrame( {'実測値':Y, '予測値':Y_pred}, columns=['実測値', '予測値'] ) # データフレームに実測値、予測結果を格納

display( results ) # 画面出力

In [ ]:
### 予測結果の確率を出力 ###

probs = model.predict_proba(X) # モデル.predict_proba(説明変数) で モデルに説明変数を入力したときの予測「確率」を抽出
probs = pd.DataFrame(probs)    # データフレーム型に変換
display( probs ) # 画面出力

In [ ]:
### 予測結果と確率の確認 ###
results['確率(Prob)'] = probs[1]     # 参考データとしてresultsのデータフレームにくっつける

display(results)

### 精度指標の確認

In [ ]:
## 混同行列、各種精度指標の可視化 ###

#ライブラリ読込：モデル評価のためのライブラリ
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

#各種 精度指標の出力
print('混同行列 (confusion matrix) = \n', confusion_matrix(y_true= Y, y_pred= Y_pred, labels=[1,0]))
print('正解率 (accuracy) = ', accuracy_score(y_true= Y, y_pred= Y_pred))
print('適合率 (precision) = ', precision_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('再現率 (recall) = ', recall_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('F値 (f1 score) = ', f1_score(y_true= Y, y_pred= Y_pred, pos_label=1))


# 演習4-2 決定木分析

## 参考：決定木可視化ツールインストール関連

### Anaconda Promptでのインストールコマンド

In [ ]:
### Anaconda Promptでの決定木可視化パッケージ "dtreeviz" インストール ###
# （※Macの場合はターミナル上からでOK）

# 下記コマンドをコピペして使用
# conda install -c conda-forge dtreeviz

# ※どうしても上記でエラーが出てうまくいかない場合は、 pip install dtreeviz を用いる
#.  ただし、conda install と pip install は相性が悪い部分もあり、公式でも非推奨

### アップデート
※最新バージョンでは無い場合

In [ ]:
# dtreeviz アップデートコマンド（Anaconda Prompt経由）
# conda update -c conda-forge dtreeviz

### dtreevizのバージョン確認

In [ ]:
import dtreeviz
print(dtreeviz.__version__)

# 2023/04/18時点の最新版は 2.2.1

### ツール動作確認コード

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

import dtreeviz

iris = load_iris()
X = iris.data
y = iris.target

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(X, y)

viz_model = dtreeviz.model(clf,
                           X_train=X, y_train=y,
                           feature_names=iris.feature_names,
                           target_name='iris',
                           class_names=iris.target_names)

# v = viz_model.view()     # render as SVG into internal object 
# v.show()                 # pop up window
# v.save("/tmp/iris.svg")  # optionally save as svg

viz_model.view()       # in notebook, displays inline

## モデルの構築

### 決定木分析モデルの構築

In [ ]:
### 決定木分析 ###
from sklearn import tree     # 機械学習パッケージに含まれた決定木分析モジュール"tree"を読込

# 説明変数のセット
X = df[ ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] ]

# 目的変数のセット
Y = df['Survived']

# 決定木モデルを作成
model = tree.DecisionTreeClassifier(max_depth=3, criterion = 'gini', random_state=0)  # インスタンス化（関数を使える状態にする）
    # 各種設定パラメータの説明
    # tree.DecisionTreeRegressor(max_depth=3) とすれば、回帰木
    # max_depth: ツリーの最大深さ
    # criterion: データの分割指標を "gini"（ジニ係数） or "entropy" （情報エントロピー）で指定
    # random_state: 乱数のシード値。指定することで再分析時の再現性を担保

    
model.fit(X, Y)                            # モデル構築（フィッティング）


### 決定木分析モデルの可視化

In [ ]:
### スマート可視化ツール "dtreeviz" を使った可視化 ###

import dtreeviz # 決定木可視化ライブラリを読み込み

viz_model = dtreeviz.model(model,
               X,                                 # 説明変数
               Y,                                 # 目的変数
               feature_names = X.columns,         # 決定木の分割条件に記載する説明変数名称
               target_name = 'Survive',           # 凡例に記載する目的変数名称
               class_names = ['0:No', '1:Yes'],   # 凡例に記載するクラス名称
              ) 

# 読み込んだ決定木モデルを描画
viz_model.view(
                fontname = 'Yu Gothic',              # 文字化け防止のためのフォント設定（Windows）
#                 fontname='Hiragino Maru Gothic Pro', # for Mac
                label_fontsize = 18,                 # 説明変数の文字サイズ
                ticks_fontsize = 14,                 # 目盛り数字の文字サイズ
                title = '決定木分析結果',               # グラフタイトル
                title_fontsize = 20,                 # グラフタイトルの文字サイズ
                scale = 1.2                          # 拡大率（描画サイズを拡大・縮小）
              )


In [ ]:
### dtreevizを使わない可視化方法 ###

# 描画サイズの指定
fig, ax = plt.subplots( figsize=(40,40) )

tree.plot_tree(model,                                #描画対象の決定木モデル
               feature_names = X.columns.to_list(),  #決定木の分割条件に記載する説明変数名称
               class_names =  ['0:No', '1:Yes'],     #凡例に記載する目的変数名称
               fontsize = 12,                        #フォントサイズの調整
               filled=True,                          #各ノードの色付け有無
               rounded=True,                         #各ノードの形状（角丸四角とするか）
               proportion=True                       #比率を出力するか
              ) 

### 変数重要度一覧の出力

In [ ]:
### 変数重要度一覧 ###
features = X.columns                      # 変数名を取得
importances = model.feature_importances_  # モデルから各変数の重要度を取得
indices = np.argsort(importances)         # 並び順を定義（大きい順序）

# グラフ描画
plt.figure(figsize=(6,6)) # グラフサイズの指定
plt.barh(range(len(indices)), importances[indices], color='b', align='center') # 横棒グラフ
plt.yticks(range(len(indices)), features[indices]) # 変数名を描画
plt.show()

## モデルの評価

### 精度指標の確認

In [ ]:
## 混同行列、各種精度指標の可視化 ###

#モデルによる予測結果の取得
Y_pred = model.predict(X)

#ライブラリ読込：モデル評価のためのライブラリ
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

#各種 精度指標の出力
print('混同行列 (confusion matrix) = \n', confusion_matrix(y_true= Y, y_pred= Y_pred, labels=[1,0]))
print('正解率 (accuracy) = ', accuracy_score(y_true= Y, y_pred= Y_pred))
print('適合率 (precision) = ', precision_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('再現率 (recall) = ', recall_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('F値 (f1 score) = ', f1_score(y_true= Y, y_pred= Y_pred, pos_label=1))


### 参考：決定境界（分割境界）の可視化

In [ ]:
### まずは、変数重要度1位、2位の変数で散布図を描画（目的変数で色分け）
sns.scatterplot(x='Sex', y='Pclass', data=df, hue='Survived')

# もしくは：1位、2位の変数の自動抽出
# sns.scatterplot(x=features[indices[-1]], y=features[indices[-2]], data=df, hue='Survived')
# df[-1]でdfの最終行、df[-2]でdfの最後から2行目

In [ ]:
### 分割領域ごとに色を塗る関数 （決定境界の可視化）###
# (参考：https://tropicbird.hatenablog.com/entry/2019/09/06/154841)
    
def visualize_classifier(X, y, ax=None, cmap='rainbow'):
    ax=ax or plt.gca()

    # 学習データをプロット
    ax.scatter(X.iloc[:,0], X.iloc[:,1], c=y, s=10, cmap=cmap, clim=(y.min(),y.max()), zorder=3)
    #ax.axis('tight')
    #ax.axis('off')  # 軸ラベルやグリッド線を隠したい場合
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # グラフの体裁
    ax.set_xlabel(X.columns[0]) # X軸ラベル
    ax.set_ylabel(X.columns[1]) # Y軸ラベル
    ax.legend()                 # 凡例
    
    # 色を塗るためのメッシュデータ（説明変数）を準備
    xx,yy = np.meshgrid(np.linspace(*xlim,num=200), np.linspace(*ylim,num=200))

    # モデル構築
    model_min = tree.DecisionTreeClassifier(max_depth=3, criterion = 'gini', random_state=0)  # インスタンス化（関数を使える状態にする）
    model_min.fit(X, y)

    # メッシュデータを入力した予測結果を出力
    Z = model_min.predict( np.c_[xx.ravel(),yy.ravel()] ).reshape(xx.shape)

    # 分割領域を色分けして表示
    n_classes = len(np.unique(y))

    contours = ax.contourf(xx,yy,Z,alpha=0.3,
                        levels=np.arange(n_classes+1)-0.5,
                        cmap=cmap, clim=(y.min(),y.max()),
                        zorder=1)
    ax.set(xlim=xlim, ylim=ylim)
    

In [ ]:
### 分割条件を色分け可視化
visualize_classifier(X[ ['Sex', 'Pclass'] ], Y)

# 演習4-3 非線形カーネルサポートベクターマシン

## モデルの構築

In [ ]:
# 非線形カーネルサポートベクターマシンによるクラス分類

## モデルの構築# 非線形カーネルサポートベクタマシンのライブラリ読み込み
from sklearn.svm import SVC  #scikit-learnパッケージ内の サポートベクタマシンモジュールより、
                             #サポートベクタマシンクラス分類のクラス (Support Vector Classifier) を読み込み
    
### 説明変数・目的変数は決定木モデルと同一のため、コメントアウト ###
# 説明変数のセット
# X = df[ ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] ]

# 目的変数のセット
# Y = df['Survived']

# サポートベクタマシンモデル構築
model = SVC()    #インスタンス化（関数を使える状態にする）
            #パラメータの説明：
            # kernel: 非線形変換させるカーネル関数の指定 (kernel ='rbf' / 'linear' / 'poly' / 'sigmoid' / 'precomputed')
            #         何も指定しなければ、デフォルト値の 'rbf'（放射基底関数: Radial Basis Function）
            
model.fit(X, Y)  # サポートベクタマシン実行

## モデルの評価

In [ ]:
# モデル予測値の出力
Y_pred = model.predict(X)

In [ ]:
## 混同行列、各種精度指標の可視化 ###

#ライブラリ読込：モデル評価のためのライブラリ
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

#各種 精度指標の出力
print('混同行列 (confusion matrix) = \n', confusion_matrix(y_true= Y, y_pred= Y_pred, labels=[1,0]))
print('正解率 (accuracy) = ', accuracy_score(y_true= Y, y_pred= Y_pred))
print('適合率 (precision) = ', precision_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('再現率 (recall) = ', recall_score(y_true= Y, y_pred= Y_pred, pos_label=1))
print('F値 (f1 score) = ', f1_score(y_true= Y, y_pred= Y_pred, pos_label=1))
